# Route4Me

Evaluation base line

- [Documentation](https://route4me.io/docs/)
- [Python SDK](https://github.com/route4me/route4me-python-sdk)

## Import

In [1]:
# Local modules
from modules import credentials, utilities

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import lxml

import time
import ast

import requests
from IPython.display import JSON

import plotly.express as px
import plotly.graph_objects as go

import polyline
import folium

# Open Calgary API
from sodapy import Socrata

import requests

from route4me import Route4Me
from route4me.constants import (
    ALGORITHM_TYPE, DISTANCE_UNIT, TRAVEL_MODE, OPTIMIZE, DEVICE_TYPE, METRIC
)

## Load data

In [2]:
gdf = utilities.load_data()
gdf

,address,longitude,latitude,geometry
0,5532 DALWOOD WY NW,-114.162743,51.114109,POINT (-114.16274 51.11411)
1,321 MAGNOLIA HE SE,-113.914848,50.891335,POINT (-113.91485 50.89134)
2,165 CRANFIELD GR SE,-113.981177,50.888362,POINT (-113.98118 50.88836)
3,2 SAGE VALLEY ME NW,-114.150439,51.178209,POINT (-114.15044 51.17821)
4,714 12 ST SE,-114.031285,51.041928,POINT (-114.03129 51.04193)
...,...,...,...,...
1995,5715 36 ST SE,-113.981822,51.001674,POINT (-113.98182 51.00167)
1996,2028 URSENBACH RD NW,-114.136418,51.070260,POINT (-114.13642 51.07026)
1997,23 ROYAL OAK PZ NW,-114.217089,51.150980,POINT (-114.21709 51.15098)
1998,8045 48 ST SE,-113.964404,50.980330,POINT (-113.96440 50.98033)


In [5]:
gdf2 = gdf[['address']] + ' Calgary, Alberta, Canada'
# gdf2.to_csv('r4m_addresses.csv')

## Prep data for r4m online

In [2]:
a = pd.read_csv(
    '../data/openYYCtestdata.csv',
    usecols=[
        'address',
        'street_name',
        'street_type',
        'street_quad',
        'house_number',
        'house_alpha'
    ]
)

In [3]:
a['street_type_long'] = a['street_type'].map(utilities.YYC_STREET_TYPEs).str.upper()
a

,address,street_name,street_type,street_quad,house_number,house_alpha,street_type_long
0,5532 DALWOOD WY NW,DALWOOD,WY,NW,5532,NaN,WAY
1,321 MAGNOLIA HE SE,MAGNOLIA,HE,SE,321,NaN,HEATH
2,165 CRANFIELD GR SE,CRANFIELD,GR,SE,165,NaN,GREEN
3,2 SAGE VALLEY ME NW,SAGE VALLEY,ME,NW,2,NaN,MEWS
4,714 12 ST SE,12,ST,SE,714,NaN,STREET
...,...,...,...,...,...,...,...
1995,5715 36 ST SE,36,ST,SE,5715,NaN,STREET
1996,2028 URSENBACH RD NW,URSENBACH,RD,NW,2028,NaN,ROAD
1997,23 ROYAL OAK PZ NW,ROYAL OAK,PZ,NW,23,NaN,PLAZA
1998,8045 48 ST SE,48,ST,SE,8045,NaN,STREET


In [4]:
a[a['street_type_long'].isna()]

,address,street_name,street_type,street_quad,house_number,house_alpha,street_type_long


In [5]:
a[['city', 'province', 'country']] = ['CALGARY', 'AB', 'CA']
a

,address,street_name,street_type,street_quad,house_number,house_alpha,street_type_long,city,province,country
0,5532 DALWOOD WY NW,DALWOOD,WY,NW,5532,NaN,WAY,CALGARY,AB,CA
1,321 MAGNOLIA HE SE,MAGNOLIA,HE,SE,321,NaN,HEATH,CALGARY,AB,CA
2,165 CRANFIELD GR SE,CRANFIELD,GR,SE,165,NaN,GREEN,CALGARY,AB,CA
3,2 SAGE VALLEY ME NW,SAGE VALLEY,ME,NW,2,NaN,MEWS,CALGARY,AB,CA
4,714 12 ST SE,12,ST,SE,714,NaN,STREET,CALGARY,AB,CA
...,...,...,...,...,...,...,...,...,...,...
1995,5715 36 ST SE,36,ST,SE,5715,NaN,STREET,CALGARY,AB,CA
1996,2028 URSENBACH RD NW,URSENBACH,RD,NW,2028,NaN,ROAD,CALGARY,AB,CA
1997,23 ROYAL OAK PZ NW,ROYAL OAK,PZ,NW,23,NaN,PLAZA,CALGARY,AB,CA
1998,8045 48 ST SE,48,ST,SE,8045,NaN,STREET,CALGARY,AB,CA


In [7]:
a['house_number_long'] = a[~a['house_alpha'].isna()][['house_number', 'house_alpha']].astype(str).agg(''.join, axis=1)
a.loc[a[a['house_alpha'].isna()].index, 'house_number_long'] = a['house_number']
a

,address,street_name,street_type,street_quad,house_number,house_alpha,street_type_long,city,province,country,house_number_long
0,5532 DALWOOD WY NW,DALWOOD,WY,NW,5532,NaN,WAY,CALGARY,AB,CA,5532
1,321 MAGNOLIA HE SE,MAGNOLIA,HE,SE,321,NaN,HEATH,CALGARY,AB,CA,321
2,165 CRANFIELD GR SE,CRANFIELD,GR,SE,165,NaN,GREEN,CALGARY,AB,CA,165
3,2 SAGE VALLEY ME NW,SAGE VALLEY,ME,NW,2,NaN,MEWS,CALGARY,AB,CA,2
4,714 12 ST SE,12,ST,SE,714,NaN,STREET,CALGARY,AB,CA,714
...,...,...,...,...,...,...,...,...,...,...,...
1995,5715 36 ST SE,36,ST,SE,5715,NaN,STREET,CALGARY,AB,CA,5715
1996,2028 URSENBACH RD NW,URSENBACH,RD,NW,2028,NaN,ROAD,CALGARY,AB,CA,2028
1997,23 ROYAL OAK PZ NW,ROYAL OAK,PZ,NW,23,NaN,PLAZA,CALGARY,AB,CA,23
1998,8045 48 ST SE,48,ST,SE,8045,NaN,STREET,CALGARY,AB,CA,8045


In [8]:
a['address_long'] = a[['house_number_long', 'street_name', 'street_type_long', 'street_quad']].astype(str).agg(' '.join, axis=1)
a['address_full'] = a[['address_long', 'city', 'province', 'country']].agg(', '.join, axis=1)
a

,address,street_name,street_type,street_quad,house_number,house_alpha,street_type_long,city,province,country,house_number_long,address_long,address_full
0,5532 DALWOOD WY NW,DALWOOD,WY,NW,5532,NaN,WAY,CALGARY,AB,CA,5532,5532 DALWOOD WAY NW,"5532 DALWOOD WAY NW, CALGARY, AB, CA"
1,321 MAGNOLIA HE SE,MAGNOLIA,HE,SE,321,NaN,HEATH,CALGARY,AB,CA,321,321 MAGNOLIA HEATH SE,"321 MAGNOLIA HEATH SE, CALGARY, AB, CA"
2,165 CRANFIELD GR SE,CRANFIELD,GR,SE,165,NaN,GREEN,CALGARY,AB,CA,165,165 CRANFIELD GREEN SE,"165 CRANFIELD GREEN SE, CALGARY, AB, CA"
3,2 SAGE VALLEY ME NW,SAGE VALLEY,ME,NW,2,NaN,MEWS,CALGARY,AB,CA,2,2 SAGE VALLEY MEWS NW,"2 SAGE VALLEY MEWS NW, CALGARY, AB, CA"
4,714 12 ST SE,12,ST,SE,714,NaN,STREET,CALGARY,AB,CA,714,714 12 STREET SE,"714 12 STREET SE, CALGARY, AB, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5715 36 ST SE,36,ST,SE,5715,NaN,STREET,CALGARY,AB,CA,5715,5715 36 STREET SE,"5715 36 STREET SE, CALGARY, AB, CA"
1996,2028 URSENBACH RD NW,URSENBACH,RD,NW,2028,NaN,ROAD,CALGARY,AB,CA,2028,2028 URSENBACH ROAD NW,"2028 URSENBACH ROAD NW, CALGARY, AB, CA"
1997,23 ROYAL OAK PZ NW,ROYAL OAK,PZ,NW,23,NaN,PLAZA,CALGARY,AB,CA,23,23 ROYAL OAK PLAZA NW,"23 ROYAL OAK PLAZA NW, CALGARY, AB, CA"
1998,8045 48 ST SE,48,ST,SE,8045,NaN,STREET,CALGARY,AB,CA,8045,8045 48 STREET SE,"8045 48 STREET SE, CALGARY, AB, CA"


In [11]:
a.iloc[:999, :].to_csv('r4m_addresses.csv', columns=['address_long', 'city', 'province', 'country'], index=False)

In [3]:
# Set up r4m API instance
r4m = Route4Me(credentials.R4M_API_Key)

## Geocoding

In [83]:
# Example
addresses = [{'address': '455 S 4th St, Louisville, KY 40202'},
             {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214'},
             {'address': '1407 MCCOY, Louisville, KY, 40215'},
             {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215'},
             {'address': '730 CECIL AVENUE, Louisville, KY, 40211'}, ]
for address in addresses:
    print('Original Address: {0}'.format(address))
    geocode_error, address = r4m.address.fix_geocode(address)
    if geocode_error:
        print('Geocoding Error: {0}'.format(geocode_error))
    print('Geocoded Address: {0}'.format(address))

Original Address: {'address': '455 S 4th St, Louisville, KY 40202'}
Geocoded Address: {'address': '455 S 4th St, Louisville, KY 40202', 'lat': 38.251373, 'lng': -85.756971}
Original Address: {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214'}
Geocoded Address: {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214', 'lat': 38.141597, 'lng': -85.793871}
Original Address: {'address': '1407 MCCOY, Louisville, KY, 40215'}
Geocoded Address: {'address': '1407 MCCOY, Louisville, KY, 40215', 'lat': 38.202454, 'lng': -85.786505}
Original Address: {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215'}
Geocoded Address: {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215', 'lat': 38.178861, 'lng': -85.775014}
Original Address: {'address': '730 CECIL AVENUE, Louisville, KY, 40211'}
Geocoded Address: {'address': '730 CECIL AVENUE, Louisville, KY, 40211', 'lat': 38.248689, 'lng': -85.821064}


In [120]:
addresses = (gdf['address'][:10] + ' Calgary Alberta Canada').str.cat(sep='\n')
addresses

'5532 DALWOOD WY NW Calgary Alberta Canada\n321 MAGNOLIA HE SE Calgary Alberta Canada\n165 CRANFIELD GR SE Calgary Alberta Canada\n2 SAGE VALLEY ME NW Calgary Alberta Canada\n714 12 ST SE Calgary Alberta Canada\n103 DEERMONT WY SE Calgary Alberta Canada\n56 TEMPLEGREEN RD NE Calgary Alberta Canada\n294 MAHOGANY PL SE Calgary Alberta Canada\n59 SANDPIPER WY NW Calgary Alberta Canada\n1925 10A ST SW Calgary Alberta Canada'

In [127]:
addresses = '5532 DALWOOD WAY NW Calgary Alberta Canada\n321 MAGNOLIA HEATH SE Calgary Alberta Canada\n165 CRANFIELD GREEN SE Calgary Alberta Canada\n2 SAGE VALLEY MEWS NW Calgary Alberta Canada\n714 12 STREET SE Calgary Alberta Canada\n103 DEERMONT WAY SE Calgary Alberta Canada\n56 TEMPLEGREEN ROAD NE Calgary Alberta Canada\n294 MAHOGANY PLACE SE Calgary Alberta Canada\n59 SANDPIPER WAY NW Calgary Alberta Canada\n1925 10A STREET SW Calgary Alberta Canada'

In [126]:
addresses =  (gdf[['address']][:10] + ' Calgary Alberta Canada').to_json(orient='records')
addresses

'[{"address":"5532 DALWOOD WY NW Calgary Alberta Canada"},{"address":"321 MAGNOLIA HE SE Calgary Alberta Canada"},{"address":"165 CRANFIELD GR SE Calgary Alberta Canada"},{"address":"2 SAGE VALLEY ME NW Calgary Alberta Canada"},{"address":"714 12 ST SE Calgary Alberta Canada"},{"address":"103 DEERMONT WY SE Calgary Alberta Canada"},{"address":"56 TEMPLEGREEN RD NE Calgary Alberta Canada"},{"address":"294 MAHOGANY PL SE Calgary Alberta Canada"},{"address":"59 SANDPIPER WY NW Calgary Alberta Canada"},{"address":"1925 10A ST SW Calgary Alberta Canada"}]'

In [114]:
addresses = [
    {"address":"5532 DALWOOD WAY NW Calgary Alberta Canada"},
    {"address":"321 MAGNOLIA HEATH SE Calgary Alberta Canada"},
    {"address":"165 CRANFIELD GREEN SE Calgary Alberta Canada"},
    {"address":"2 SAGE VALLEY MEWS NW Calgary Alberta Canada"},
    {"address":"714 12 STREET SE Calgary Alberta Canada"},
    {"address":"103 DEERMONT WAY SE Calgary Alberta Canada"},
    {"address":"56 TEMPLEGREEN ROAD NE Calgary Alberta Canada"},
    {"address":"294 MAHOGANY PLACE SE Calgary Alberta Canada"},
    {"address":"59 SANDPIPER WAY NW Calgary Alberta Canada"},
    {"address":"1925 10A STREET SW Calgary Alberta Canada"}]

In [128]:
url = 'https://api.route4me.com/api/geocoder.php'
params={
    'api_key': credentials.R4M_API_Key,
    'addresses': addresses,
    'strExportFormat': 'json'
}

response = requests.post(url=url, params=params)

In [129]:
pd.read_xml(response.text)

,destination,lat,lng,type,normalized,confidence,original
0,"5532 Dalwood Way NW, Calgary, AB T3A 1S7, Canada",51.114209,-114.162902,premise,5532 dalwood way nw calgary alberta canada 321...,high,5532 DALWOOD WAY NW Calgary Alberta Canada 321...
1,"165 Cranfield Green SE, Calgary, AB T3M 1C6, C...",50.888439,-113.981061,premise,5532 dalwood way nw calgary alberta canada 321...,high,5532 DALWOOD WAY NW Calgary Alberta Canada 321...
2,"2 Cranfield Green SE, Calgary, AB T3M, Canada",50.890584,-113.979557,street_address,5532 dalwood way nw calgary alberta canada 321...,medium,5532 DALWOOD WAY NW Calgary Alberta Canada 321...
3,"2 Sage Valley Mews NW, Calgary, AB T3R 0H7, Ca...",51.178146,-114.150626,street_address,5532 dalwood way nw calgary alberta canada 321...,high,5532 DALWOOD WAY NW Calgary Alberta Canada 321...


In [124]:
response.text

'<?xml version="1.0" encoding="UTF-8" ?><destinations>\n\t<destination  destination="5532 Dalwood Way NW, Calgary, AB T3A 1S7, Canada" lat="51.114209" lng="-114.162902" type="premise" normalized="5532 dalwood way nw calgary alberta canada\n321 magnolia heath se calgary alberta canada\n165 cranfield green se calgary alberta canada\n2 sage valley mews nw calgary alberta canada\n714 12 street se calgary alberta canada\n103 deermont way se calgary alberta canada\n56 templegreen road ne calgary alberta canada\n294 mahogany place se calgary alberta canada\n59 sandpiper way nw calgary alberta canada\n1925 10a street sw calgary alberta canada" confidence="high" original="5532 DALWOOD WaY NW Calgary Alberta Canada\n321 MAGNOLIA HEATH SE Calgary Alberta Canada\n165 CRANFIELD GREEN SE Calgary Alberta Canada\n2 SAGE VALLEY MEWS NW Calgary Alberta Canada\n714 12 STREET SE Calgary Alberta Canada\n103 DEERMONT WAY SE Calgary Alberta Canada\n56 TEMPLEGREEN ROAD NE Calgary Alberta Canada\n294 MAHOGANY 

In [43]:
response = r4m.address.geocode(
    addresses=addresses,
    format='json'
)

In [47]:
response = r4m.address.geocode(
    addresses='165 CRANFIELD GR SE Calgary Alberta Canada',
    format='json'
)

In [48]:
response

b'[{"address":"165 Cranford Dr SE, Calgary, AB T3M 0J1, Canada","lat":50.876892,"lng":-113.969611,"type":"street_address","normalized":"165 cranfield gr se calgary alberta canada","confidence":"high","original":"165 CRANFIELD GR SE Calgary Alberta Canada"}]'

In [44]:
pd.json_normalize(ast.literal_eval(response.decode('utf-8')))

,address,lat,lng,type,normalized,confidence,original
0,"5532 Dalwood Way NW, Calgary, AB T3A 1S7, Canada",51.114209,-114.162902,premise,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
1,"714 12 Street SE, Calgary, AB T2G 3H5, Canada",51.042366,-114.032847,street_address,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
2,"321 Magnolia He SE, Calgary, AB T3M 2B6, Canada",50.891304,-113.914862,street_address,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
3,"2 Sage Valley Dr NW, Calgary, AB T3R, Canada",51.182912,-114.147397,street_address,5532 dalwood wy nw calgary alberta canada\n321...,medium,5532 DALWOOD WY NW Calgary Alberta Canada\n321...


^ 6 addresses are missing

In [46]:
JSON(ast.literal_eval(response.decode('utf-8')))

<IPython.core.display.JSON object>

In [21]:
route = r4m.route
response = route.get_routes(limit=1, offset=0)
if isinstance(response, dict) and 'errors' in response.keys():
    print('. '.join(response['errors']))
else:
    route_id = response[0]['route_id']
    print('Route ID: {}'.format(route_id))
    response = route.get_route(route_id=route_id)
    if isinstance(response, dict) and 'errors' in response.keys():
        print('. '.join(response['errors']))
    else:
        print('Optimization Problem ID: {}'.format(
            response['optimization_problem_id']
        ))
        for address in response['addresses']:
            print('\t\t\tID: {0} - Address: {1}'.format(address['route_destination_id'], address['address']))

Route ID: 9689A6C2946D106D1BD0FB0B911A4F70
Optimization Problem ID: F023857CD5F487810739CD6B025C2445
			ID: 763102910 - Address: 754 5th Ave New York, NY 10019
			ID: 763102911 - Address: 717 5th Ave New York, NY 10022
			ID: 763102912 - Address: 888 Madison Ave New York, NY 10014
			ID: 763102913 - Address: 1011 Madison Ave New York, NY 10075
			ID: 763102914 - Address: 440 Columbus Ave New York, NY 10024
			ID: 763102915 - Address: 324 Columbus Ave #1 New York, NY 10023
			ID: 763102916 - Address: 110 W End Ave New York, NY 10023
			ID: 763102917 - Address: 555 W 57th St New York, NY 10019
			ID: 763102918 - Address: 57 W 57th St New York, NY 10019
			ID: 763102910 - Address: 754 5th Ave New York, NY 10019


In [31]:
optimization = r4m.optimization
address = r4m.address
optimization.algorithm_type(ALGORITHM_TYPE.CVRP_TW_SD)
optimization.share_route(0)
optimization.store_route(0)
optimization.route_time(0)
optimization.parts(4)
optimization.route_max_duration(86400)
optimization.vehicle_max_distance_mi(10000)
optimization.route_name('Single Depot, Multiple Driver Time Window')
optimization.optimize(OPTIMIZE.TIME)
optimization.distance_unit(DISTANCE_UNIT.MI)
optimization.device_type(DEVICE_TYPE.WEB)
optimization.travel_mode(TRAVEL_MODE.DRIVING)

address.add_address(
    address="455 S 4th St, Louisville, KY 40202",
    lat=38.251698,
    lng=-85.757308,
    is_depot=1
)
address.add_address(
    address="1604 PARKRIDGE PKWY, Louisville, KY, 40214",
    lat=38.141598,
    lng=-85.793846,
    is_depot=0,
    time=300,
    time_window_start=29400,
    time_window_end=30000
)
address.add_address(
    address="1407 MCCOY, Louisville, KY, 40215",
    lat=38.202496,
    lng=-85.786514,
    is_depot=0,
    time=300,
    time_window_start=30000,
    time_window_end=30600
)
address.add_address(
    address="4805 BELLEVUE AVE, Louisville, KY, 40215",
    lat=38.178844,
    lng=-85.774864,
    is_depot=0,
    time=300,
    time_window_start=30600,
    time_window_end=31200
)
address.add_address(
    address="730 CECIL AVENUE, Louisville, KY, 40211",
    lat=38.248684,
    lng=-85.821121,
    is_depot=0,
    time=300,
    time_window_start=31200,
    time_window_end=31800
)
address.add_address(
    address="650 SOUTH 29TH ST UNIT 315, Louisville, KY, 40211",
    lat=38.251923,
    lng=-85.800034,
    is_depot=0,
    time=300,
    time_window_start=31800,
    time_window_end=32400
)
address.add_address(
    address="4629 HILLSIDE DRIVE, Louisville, KY, 40216",
    lat=38.176067,
    lng=-85.824638,
    is_depot=0,
    time=300,
    time_window_start=32400,
    time_window_end=33000
)
address.add_address(
    address="4738 BELLEVUE AVE, Louisville, KY, 40215",
    lat=38.179806,
    lng=-85.775558,
    is_depot=0,
    time=300,
    time_window_start=33000,
    time_window_end=33600
)
address.add_address(
    address="318 SO. 39TH STREET, Louisville, KY, 40212",
    lat=38.259335,
    lng=-85.815094,
    is_depot=0,
    time=300,
    time_window_start=33600,
    time_window_end=34200
)
address.add_address(
    address="1324 BLUEGRASS AVE, Louisville, KY, 40215",
    lat=38.179253,
    lng=-85.785118,
    is_depot=0,
    time=300,
    time_window_start=34200,
    time_window_end=34800
)
address.add_address(
    address="7305 ROYAL WOODS DR, Louisville, KY, 40214",
    lat=38.162472,
    lng=-85.792854,
    is_depot=0,
    time=300,
    time_window_start=34800,
    time_window_end=35400
)
address.add_address(
    address="1661 W HILL ST, Louisville, KY, 40210",
    lat=38.229584,
    lng=-85.783966,
    is_depot=0,
    time=300,
    time_window_start=35400,
    time_window_end=36000
)
address.add_address(
    address="3222 KINGSWOOD WAY, Louisville, KY, 40216",
    lat=38.210606,
    lng=-85.822594,
    is_depot=0,
    time=300,
    time_window_start=36000,
    time_window_end=36600
)
address.add_address(
    address="1922 PALATKA RD, Louisville, KY, 40214",
    lat=38.153767,
    lng=-85.796783,
    is_depot=0,
    time=300,
    time_window_start=36600,
    time_window_end=37200
)
address.add_address(
    address="1314 SOUTH 26TH STREET, Louisville, KY, 40210",
    lat=38.235847,
    lng=-85.796852,
    is_depot=0,
    time=300,
    time_window_start=37200,
    time_window_end=37800
)
address.add_address(
    address="2135 MCCLOSKEY AVENUE, Louisville, KY, 40210",
    lat=38.218662,
    lng=-85.789032,
    is_depot=0,
    time=300,
    time_window_start=37800,
    time_window_end=38400
)
address.add_address(
    address="1409 PHYLLIS AVE, Louisville, KY, 40215",
    lat=38.206154,
    lng=-85.781387,
    is_depot=0,
    time=300,
    time_window_start=38400,
    time_window_end=39000
)
address.add_address(
    address="4504 SUNFLOWER AVE, Louisville, KY, 40216",
    lat=38.187511,
    lng=-85.839149,
    is_depot=0,
    time=300,
    time_window_start=39000,
    time_window_end=39600
)
address.add_address(
    address="2512 GREENWOOD AVE, Louisville, KY, 40210",
    lat=38.241405,
    lng=-85.795059,
    is_depot=0,
    time=300,
    time_window_start=39600,
    time_window_end=40200
)
address.add_address(
    address="5500 WILKE FARM AVE, Louisville, KY, 40216",
    lat=38.166065,
    lng=-85.863319,
    is_depot=0,
    time=300,
    time_window_start=40200,
    time_window_end=40800
)
address.add_address(
    address="3640 LENTZ AVE, Louisville, KY, 40215",
    lat=38.193283,
    lng=-85.786201,
    is_depot=0,
    time=300,
    time_window_start=40800,
    time_window_end=41400
)
address.add_address(
    address="1020 BLUEGRASS AVE, Louisville, KY, 40215",
    lat=38.17952,
    lng=-85.780037,
    is_depot=0,
    time=300,
    time_window_start=41400,
    time_window_end=42000
)
address.add_address(
    address="123 NORTH 40TH ST, Louisville, KY, 40212",
    lat=38.26498,
    lng=-85.814156,
    is_depot=0,
    time=300,
    time_window_start=42000,
    time_window_end=42600
)
address.add_address(
    address="7315 ST ANDREWS WOODS CIRCLE UNIT 104, Louisville, KY, 40214",
    lat=38.151072,
    lng=-85.802867,
    is_depot=0,
    time=300,
    time_window_start=42600,
    time_window_end=43200
)
address.add_address(
    address="3210 POPLAR VIEW DR, Louisville, KY, 40216",
    lat=38.182594,
    lng=-85.849937,
    is_depot=0,
    time=300,
    time_window_start=43200,
    time_window_end=43800
)
address.add_address(
    address="4519 LOUANE WAY, Louisville, KY, 40216",
    lat=38.1754,
    lng=-85.811447,
    is_depot=0,
    time=300,
    time_window_start=43800,
    time_window_end=44400
)
address.add_address(
    address="6812 MANSLICK RD, Louisville, KY, 40214",
    lat=38.161839,
    lng=-85.798279,
    is_depot=0,
    time=300,
    time_window_start=44400,
    time_window_end=45000
)
address.add_address(
    address="1524 HUNTOON AVENUE, Louisville, KY, 40215",
    lat=38.172031,
    lng=-85.788353,
    is_depot=0,
    time=300,
    time_window_start=45000,
    time_window_end=45600
)
address.add_address(
    address="1307 LARCHMONT AVE, Louisville, KY, 40215",
    lat=38.209663,
    lng=-85.779816,
    is_depot=0,
    time=300,
    time_window_start=45600,
    time_window_end=46200
)
address.add_address(
    address="434 N 26TH STREET #2, Louisville, KY, 40212",
    lat=38.26844,
    lng=-85.791962,
    is_depot=0,
    time=300,
    time_window_start=46200,
    time_window_end=46800
)
address.add_address(
    address="678 WESTLAWN ST, Louisville, KY, 40211",
    lat=38.250397,
    lng=-85.80629,
    is_depot=0,
    time=300,
    time_window_start=46800,
    time_window_end=47400
)
address.add_address(
    address="2308 W BROADWAY, Louisville, KY, 40211",
    lat=38.248882,
    lng=-85.790421,
    is_depot=0,
    time=300,
    time_window_start=47400,
    time_window_end=48000
)
address.add_address(
    address="2332 WOODLAND AVE, Louisville, KY, 40210",
    lat=38.233579,
    lng=-85.794257,
    is_depot=0,
    time=300,
    time_window_start=48000,
    time_window_end=48600
)
address.add_address(
    address="1706 WEST ST. CATHERINE, Louisville, KY, 40210",
    lat=38.239697,
    lng=-85.783928,
    is_depot=0,
    time=300,
    time_window_start=48600,
    time_window_end=49200
)
address.add_address(
    address="1699 WATHEN LN, Louisville, KY, 40216",
    lat=38.216465,
    lng=-85.792397,
    is_depot=0,
    time=300,
    time_window_start=49200,
    time_window_end=49800
)
address.add_address(
    address="2416 SUNSHINE WAY, Louisville, KY, 40216",
    lat=38.186245,
    lng=-85.831787,
    is_depot=0,
    time=300,
    time_window_start=49800,
    time_window_end=50400
)
address.add_address(
    address="6925 MANSLICK RD, Louisville, KY, 40214",
    lat=38.158466,
    lng=-85.798355,
    is_depot=0,
    time=300,
    time_window_start=50400,
    time_window_end=51000
)
address.add_address(
    address="2707 7TH ST, Louisville, KY, 40215",
    lat=38.212438,
    lng=-85.785082,
    is_depot=0,
    time=300,
    time_window_start=51000,
    time_window_end=51600
)
address.add_address(
    address="2014 KENDALL LN, Louisville, KY, 40216",
    lat=38.179394,
    lng=-85.826668,
    is_depot=0,
    time=300,
    time_window_start=51600,
    time_window_end=52200
)
address.add_address(
    address="612 N 39TH ST, Louisville, KY, 40212",
    lat=38.273354,
    lng=-85.812012,
    is_depot=0,
    time=300,
    time_window_start=52200,
    time_window_end=52800
)
address.add_address(
    address="2215 ROWAN ST, Louisville, KY, 40212",
    lat=38.261703,
    lng=-85.786781,
    is_depot=0,
    time=300,
    time_window_start=52800,
    time_window_end=53400
)
address.add_address(
    address="1826 W. KENTUCKY ST, Louisville, KY, 40210",
    lat=38.241611,
    lng=-85.78653,
    is_depot=0,
    time=300,
    time_window_start=53400,
    time_window_end=54000
)
address.add_address(
    address="1810 GREGG AVE, Louisville, KY, 40210",
    lat=38.224716,
    lng=-85.796211,
    is_depot=0,
    time=300,
    time_window_start=54000,
    time_window_end=54600
)
address.add_address(
    address="4103 BURRRELL DRIVE, Louisville, KY, 40216",
    lat=38.191753,
    lng=-85.825836,
    is_depot=0,
    time=300,
    time_window_start=54600,
    time_window_end=55200
)
address.add_address(
    address="359 SOUTHWESTERN PKWY, Louisville, KY, 40212",
    lat=38.259903,
    lng=-85.823463,
    is_depot=0,
    time=300,
    time_window_start=55200,
    time_window_end=55800
)
address.add_address(
    address="2407 W CHESTNUT ST, Louisville, KY, 40211",
    lat=38.252781,
    lng=-85.792109,
    is_depot=0,
    time=300,
    time_window_start=55800,
    time_window_end=56400
)
address.add_address(
    address="225 S 22ND ST, Louisville, KY, 40212",
    lat=38.257616,
    lng=-85.786658,
    is_depot=0,
    time=300,
    time_window_start=56400,
    time_window_end=57000
)
address.add_address(
    address="1404 MCCOY AVE, Louisville, KY, 40215",
    lat=38.202122,
    lng=-85.786072,
    is_depot=0,
    time=300,
    time_window_start=57000,
    time_window_end=57600
)
address.add_address(
    address="117 FOUNT LANDING CT, Louisville, KY, 40212",
    lat=38.270061,
    lng=-85.799438,
    is_depot=0,
    time=300,
    time_window_start=57600,
    time_window_end=58200
)
address.add_address(
    address="5504 SHOREWOOD DRIVE, Louisville, KY, 40214",
    lat=38.145851,
    lng=-85.7798,
    is_depot=0,
    time=300,
    time_window_start=58200,
    time_window_end=58800
)

response = r4m.run_optimization()
print('Optimization Link: {}'.format(response['links']['view']))
for i, route in enumerate(response['routes']):
    print('\t{0}\tRoute Link: {1}'.format(i + 1, route['links']['route']))
    for address in route['addresses']:
        print('\t\t\tAddress: {0}'.format(address['address']))

Optimization Link: https://api.route4me.com/api.v4/optimization_problem.php?optimization_problem_id=3A0C915DBE468924DB6B45624EB676C4&api_key=26B8C55FF6290B3909BD4C07402C8320&member_id=2232353
	1	Route Link: https://api.route4me.com/api.v4/route.php?route_id=39E7BB388C877482D6B6CC31B6F9CC5F&api_key=26B8C55FF6290B3909BD4C07402C8320&member_id=2232353&device_tracking_history=0&original=0&notes=0
			Address: 455 S 4th St, Louisville, KY 40202
			Address: 2707 7TH ST, Louisville, KY, 40215
			Address: 1404 MCCOY AVE, Louisville, KY, 40215
			Address: 1407 MCCOY, Louisville, KY, 40215
			Address: 1407 MCCOY, Louisville, KY, 40215
			Address: 4629 HILLSIDE DRIVE, Louisville, KY, 40216
			Address: 4629 HILLSIDE DRIVE, Louisville, KY, 40216
			Address: 4629 HILLSIDE DRIVE, Louisville, KY, 40216
			Address: 4738 BELLEVUE AVE, Louisville, KY, 40215
			Address: 1324 BLUEGRASS AVE, Louisville, KY, 40215
			Address: 1324 BLUEGRASS AVE, Louisville, KY, 40215
			Address: 1324 BLUEGRASS AVE, Louisville,

In [28]:
optimization = r4m.optimization
address = r4m.address
optimization.route_name('Multiple Depot, Multiple Driver, Time window')
optimization.algorithm_type(ALGORITHM_TYPE.CVRP_TW_MD)
optimization.share_route(0)
optimization.store_route(0)
optimization.device_type(DEVICE_TYPE.WEB)
optimization.distance_unit(DISTANCE_UNIT.KM)
optimization.travel_mode(TRAVEL_MODE.DRIVING)
optimization.metric(METRIC.ROUTE4ME_METRIC_MATRIX)
optimization.vehicle_capacity(9999)
optimization.vehicle_max_distance_mi(99999)
optimization.parts(10)
optimization.route_time(7 * 3600)
optimization.rt(1)
optimization.route_max_duration(86400)
optimization.optimize(OPTIMIZE.TIME)

address.add_address(
    address="455 S 4th St, Louisville, KY 40202",
    lat=38.251698,
    lng=-85.757308,
    is_depot=1,
    time=300
)
address.add_address(
    address="1604 PARKRIDGE PKWY, Louisville, KY, 40214",
    lat=38.141598,
    lng=-85.793846,
    is_depot=1,
    time=300
)
address.add_address(
    address="1407 MCCOY, Louisville, KY, 40215",
    lat=38.202496,
    lng=-85.786514,
    is_depot=0,
    time=300,
    time_window_start=30000,
    time_window_end=30600
)
address.add_address(
    address="4805 BELLEVUE AVE, Louisville, KY, 40215",
    lat=38.178844,
    lng=-85.774864,
    is_depot=0,
    time=300,
    time_window_start=30600,
    time_window_end=31200
)
address.add_address(
    address="730 CECIL AVENUE, Louisville, KY, 40211",
    lat=38.248684,
    lng=-85.821121,
    is_depot=0,
    time=300,
    time_window_start=31200,
    time_window_end=31800
)
address.add_address(
    address="650 SOUTH 29TH ST UNIT 315, Louisville, KY, 40211",
    lat=38.251923,
    lng=-85.800034,
    is_depot=0,
    time=300,
    time_window_start=31800,
    time_window_end=32400
)
address.add_address(
    address="4629 HILLSIDE DRIVE, Louisville, KY, 40216",
    lat=38.176067,
    lng=-85.824638,
    is_depot=0,
    time=300,
    time_window_start=32400,
    time_window_end=33000
)
address.add_address(
    address="4738 BELLEVUE AVE, Louisville, KY, 40215",
    lat=38.179806,
    lng=-85.775558,
    is_depot=0,
    time=300,
    time_window_start=33000,
    time_window_end=33600
)
address.add_address(
    address="318 SO. 39TH STREET, Louisville, KY, 40212",
    lat=38.259335,
    lng=-85.815094,
    is_depot=0,
    time=300,
    time_window_start=33600,
    time_window_end=34200
)
address.add_address(
    address="1324 BLUEGRASS AVE, Louisville, KY, 40215",
    lat=38.179253,
    lng=-85.785118,
    is_depot=0,
    time=300,
    time_window_start=34200,
    time_window_end=34800
)
address.add_address(
    address="7305 ROYAL WOODS DR, Louisville, KY, 40214",
    lat=38.162472,
    lng=-85.792854,
    is_depot=0,
    time=300,
    time_window_start=34800,
    time_window_end=35400
)
address.add_address(
    address="1661 W HILL ST, Louisville, KY, 40210",
    lat=38.229584,
    lng=-85.783966,
    is_depot=0,
    time=300,
    time_window_start=35400,
    time_window_end=36000
)
address.add_address(
    address="3222 KINGSWOOD WAY, Louisville, KY, 40216",
    lat=38.210606,
    lng=-85.822594,
    is_depot=0,
    time=300,
    time_window_start=36000,
    time_window_end=36600
)
address.add_address(
    address="1922 PALATKA RD, Louisville, KY, 40214",
    lat=38.153767,
    lng=-85.796783,
    is_depot=0,
    time=300,
    time_window_start=36600,
    time_window_end=37200
)
address.add_address(
    address="1314 SOUTH 26TH STREET, Louisville, KY, 40210",
    lat=38.235847,
    lng=-85.796852,
    is_depot=0,
    time=300,
    time_window_start=37200,
    time_window_end=37800
)
address.add_address(
    address="2135 MCCLOSKEY AVENUE, Louisville, KY, 40210",
    lat=38.218662,
    lng=-85.789032,
    is_depot=0,
    time=300,
    time_window_start=37800,
    time_window_end=38400
)
address.add_address(
    address="1409 PHYLLIS AVE, Louisville, KY, 40215",
    lat=38.206154,
    lng=-85.781387,
    is_depot=0,
    time=300,
    time_window_start=38400,
    time_window_end=39000
)
address.add_address(
    address="4504 SUNFLOWER AVE, Louisville, KY, 40216",
    lat=38.187511,
    lng=-85.839149,
    is_depot=0,
    time=300,
    time_window_start=39000,
    time_window_end=39600
)
address.add_address(
    address="2512 GREENWOOD AVE, Louisville, KY, 40210",
    lat=38.241405,
    lng=-85.795059,
    is_depot=0,
    time=300,
    time_window_start=39600,
    time_window_end=40200
)
address.add_address(
    address="5500 WILKE FARM AVE, Louisville, KY, 40216",
    lat=38.166065,
    lng=-85.863319,
    is_depot=0,
    time=300,
    time_window_start=40200,
    time_window_end=40800
)
address.add_address(
    address="3640 LENTZ AVE, Louisville, KY, 40215",
    lat=38.193283,
    lng=-85.786201,
    is_depot=0,
    time=300,
    time_window_start=40800,
    time_window_end=41400
)
address.add_address(
    address="1020 BLUEGRASS AVE, Louisville, KY, 40215",
    lat=38.17952,
    lng=-85.780037,
    is_depot=0,
    time=300,
    time_window_start=41400,
    time_window_end=42000
)
address.add_address(
    address="123 NORTH 40TH ST, Louisville, KY, 40212",
    lat=38.26498,
    lng=-85.814156,
    is_depot=0,
    time=300,
    time_window_start=42000,
    time_window_end=42600
)
address.add_address(
    address="7315 ST ANDREWS WOODS CIRCLE UNIT 104, Louisville, KY, 40214",
    lat=38.151072,
    lng=-85.802867,
    is_depot=0,
    time=300,
    time_window_start=42600,
    time_window_end=43200
)
address.add_address(
    address="3210 POPLAR VIEW DR, Louisville, KY, 40216",
    lat=38.182594,
    lng=-85.849937,
    is_depot=0,
    time=300,
    time_window_start=43200,
    time_window_end=43800
)
address.add_address(
    address="4519 LOUANE WAY, Louisville, KY, 40216",
    lat=38.1754,
    lng=-85.811447,
    is_depot=0,
    time=300,
    time_window_start=43800,
    time_window_end=44400
)
address.add_address(
    address="6812 MANSLICK RD, Louisville, KY, 40214",
    lat=38.161839,
    lng=-85.798279,
    is_depot=0,
    time=300,
    time_window_start=44400,
    time_window_end=45000
)
address.add_address(
    address="1524 HUNTOON AVENUE, Louisville, KY, 40215",
    lat=38.172031,
    lng=-85.788353,
    is_depot=0,
    time=300,
    time_window_start=45000,
    time_window_end=45600
)
address.add_address(
    address="1307 LARCHMONT AVE, Louisville, KY, 40215",
    lat=38.209663,
    lng=-85.779816,
    is_depot=0,
    time=300,
    time_window_start=45600,
    time_window_end=46200
)
address.add_address(
    address="434 N 26TH STREET #2, Louisville, KY, 40212",
    lat=38.26844,
    lng=-85.791962,
    is_depot=0,
    time=300,
    time_window_start=46200,
    time_window_end=46800
)
address.add_address(
    address="678 WESTLAWN ST, Louisville, KY, 40211",
    lat=38.250397,
    lng=-85.80629,
    is_depot=0,
    time=300,
    time_window_start=46800,
    time_window_end=47400
)
address.add_address(
    address="2308 W BROADWAY, Louisville, KY, 40211",
    lat=38.248882,
    lng=-85.790421,
    is_depot=0,
    time=300,
    time_window_start=47400,
    time_window_end=48000
)
address.add_address(
    address="2332 WOODLAND AVE, Louisville, KY, 40210",
    lat=38.233579,
    lng=-85.794257,
    is_depot=0,
    time=300,
    time_window_start=48000,
    time_window_end=48600
)
address.add_address(
    address="1706 WEST ST. CATHERINE, Louisville, KY, 40210",
    lat=38.239697,
    lng=-85.783928,
    is_depot=0,
    time=300,
    time_window_start=48600,
    time_window_end=49200
)
address.add_address(
    address="1699 WATHEN LN, Louisville, KY, 40216",
    lat=38.216465,
    lng=-85.792397,
    is_depot=0,
    time=300,
    time_window_start=49200,
    time_window_end=49800
)
address.add_address(
    address="2416 SUNSHINE WAY, Louisville, KY, 40216",
    lat=38.186245,
    lng=-85.831787,
    is_depot=0,
    time=300,
    time_window_start=49800,
    time_window_end=50400
)
address.add_address(
    address="6925 MANSLICK RD, Louisville, KY, 40214",
    lat=38.158466,
    lng=-85.798355,
    is_depot=0,
    time=300,
    time_window_start=50400,
    time_window_end=51000
)
address.add_address(
    address="2707 7TH ST, Louisville, KY, 40215",
    lat=38.212438,
    lng=-85.785082,
    is_depot=0,
    time=300,
    time_window_start=51000,
    time_window_end=51600
)
address.add_address(
    address="2014 KENDALL LN, Louisville, KY, 40216",
    lat=38.179394,
    lng=-85.826668,
    is_depot=0,
    time=300,
    time_window_start=51600,
    time_window_end=52200
)
address.add_address(
    address="612 N 39TH ST, Louisville, KY, 40212",
    lat=38.273354,
    lng=-85.812012,
    is_depot=0,
    time=300,
    time_window_start=52200,
    time_window_end=52800
)
address.add_address(
    address="2215 ROWAN ST, Louisville, KY, 40212",
    lat=38.261703,
    lng=-85.786781,
    is_depot=0,
    time=300,
    time_window_start=52800,
    time_window_end=53400
)
address.add_address(
    address="1826 W. KENTUCKY ST, Louisville, KY, 40210",
    lat=38.241611,
    lng=-85.78653,
    is_depot=0,
    time=300,
    time_window_start=53400,
    time_window_end=54000
)
address.add_address(
    address="1810 GREGG AVE, Louisville, KY, 40210",
    lat=38.224716,
    lng=-85.796211,
    is_depot=0,
    time=300,
    time_window_start=54000,
    time_window_end=54600
)
address.add_address(
    address="4103 BURRRELL DRIVE, Louisville, KY, 40216",
    lat=38.191753,
    lng=-85.825836,
    is_depot=0,
    time=300,
    time_window_start=54600,
    time_window_end=55200
)
address.add_address(
    address="359 SOUTHWESTERN PKWY, Louisville, KY, 40212",
    lat=38.259903,
    lng=-85.823463,
    is_depot=0,
    time=300,
    time_window_start=55200,
    time_window_end=55800
)
address.add_address(
    address="2407 W CHESTNUT ST, Louisville, KY, 40211",
    lat=38.252781,
    lng=-85.792109,
    is_depot=0,
    time=300,
    time_window_start=55800,
    time_window_end=56400
)
address.add_address(
    address="225 S 22ND ST, Louisville, KY, 40212",
    lat=38.257616,
    lng=-85.786658,
    is_depot=0,
    time=300,
    time_window_start=56400,
    time_window_end=57000
)
address.add_address(
    address="1404 MCCOY AVE, Louisville, KY, 40215",
    lat=38.202122,
    lng=-85.786072,
    is_depot=0,
    time=300,
    time_window_start=57000,
    time_window_end=57600
)
address.add_address(
    address="117 FOUNT LANDING CT, Louisville, KY, 40212",
    lat=38.270061,
    lng=-85.799438,
    is_depot=0,
    time=300,
    time_window_start=57600,
    time_window_end=58200
)
address.add_address(
    address="5504 SHOREWOOD DRIVE, Louisville, KY, 40214",
    lat=38.145851,
    lng=-85.7798,
    is_depot=0,
    time=300,
    time_window_start=58200,
    time_window_end=58800
)

response = r4m.run_optimization()
print('Optimization Link: {}'.format(response['links']['view']))
for i, route in enumerate(response['routes']):
    print('\t{0}\tRoute Link: {1}'.format(i + 1, route['links']['route']))
    for address in route['addresses']:
        print('\t\t\tAddress: {0}'.format(address['address']))

Optimization Link: https://api.route4me.com/api.v4/optimization_problem.php?optimization_problem_id=62259BD3DAC7B10F100040C1D7461DC7&api_key=26B8C55FF6290B3909BD4C07402C8320&member_id=2232353
	1	Route Link: https://api.route4me.com/api.v4/route.php?route_id=1AF2EC4618D6154DE731EE4D9FAAA36E&api_key=26B8C55FF6290B3909BD4C07402C8320&member_id=2232353&device_tracking_history=0&original=0&notes=0
			Address: 455 S 4th St, Louisville, KY 40202
			Address: 1661 W HILL ST, Louisville, KY, 40210
			Address: 2135 MCCLOSKEY AVENUE, Louisville, KY, 40210
			Address: 4504 SUNFLOWER AVE, Louisville, KY, 40216
			Address: 4504 SUNFLOWER AVE, Louisville, KY, 40216
			Address: 4504 SUNFLOWER AVE, Louisville, KY, 40216
			Address: 5500 WILKE FARM AVE, Louisville, KY, 40216
			Address: 5500 WILKE FARM AVE, Louisville, KY, 40216
			Address: 5500 WILKE FARM AVE, Louisville, KY, 40216
			Address: 3210 POPLAR VIEW DR, Louisville, KY, 40216
			Address: 3210 POPLAR VIEW DR, Louisville, KY, 40216
			Address: 32

In [3]:
optimization = r4m.optimization
address = r4m.address
optimization.algorithm_type(ALGORITHM_TYPE.TSP)
optimization.share_route(0)
optimization.store_route(0)
optimization.route_time(7 * 600)
optimization.rt(True)
optimization.route_max_duration(86400)
optimization.route_name('Single Driver Round Trip')
optimization.optimize(OPTIMIZE.TIME)
optimization.distance_unit(DISTANCE_UNIT.MI)
optimization.travel_mode(TRAVEL_MODE.DRIVING)
address.add_address(
    address='754 5th Ave New York, NY 10019',
    lat=40.7636197,
    lng=-73.9744388,
    alias='Bergdorf Goodman',
    is_depot=True,
    time=0
)
address.add_address(
    address='717 5th Ave New York, NY 10022',
    lat=40.7669692,
    lng=-73.9693864,
    alias='Giorgio Armani',
    time=60
)
address.add_address(
    address='888 Madison Ave New York, NY 10014',
    lat=40.7715154,
    lng=-73.9669241,
    alias='Ralph Lauren Women\'s and Home',
    time=60
)
address.add_address(
    address='1011 Madison Ave New York, NY 10075',
    lat=40.7772129,
    lng=-73.9669,
    alias='Yigal Azrou\u00ebl',
    time=60
)
address.add_address(
    address='440 Columbus Ave New York, NY 10024',
    lat=40.7808364,
    lng=-73.9732729,
    alias='Frank Stella Clothier',
    time=60
)
address.add_address(
    address='324 Columbus Ave #1 New York, NY 10023',
    lat=40.7803123,
    lng=-73.9793079,
    alias='Liana',
    time=60
)
address.add_address(
    address='110 W End Ave New York, NY 10023',
    lat=40.7753077,
    lng=-73.9861529,
    alias='Toga Bike Shop',
    time=60
)
address.add_address(
    address='555 W 57th St New York, NY 10019',
    lat=40.7718005,
    lng=-73.9897716,
    alias='BMW of Manhattan',
    time=60
)
address.add_address(
    address='57 W 57th St New York, NY 10019',
    lat=40.7558695,
    lng=-73.9862019,
    alias='Verizon Wireless',
    time=60
)

response = r4m.run_optimization()
print("Optimization Problem ID: {}".format(response.get("optimization_problem_id")))
for a in response.get("addresses", []):
    print('{} - {} - {}'.format(a.get("sequence_no"), a.get("alias"), a.get("address")))

Optimization Problem ID: F023857CD5F487810739CD6B025C2445
0 - Bergdorf Goodman - 754 5th Ave New York, NY 10019
1 - Giorgio Armani - 717 5th Ave New York, NY 10022
2 - Ralph Lauren Women's and Home - 888 Madison Ave New York, NY 10014
3 - Yigal Azrouël - 1011 Madison Ave New York, NY 10075
4 - Frank Stella Clothier - 440 Columbus Ave New York, NY 10024
5 - Liana - 324 Columbus Ave #1 New York, NY 10023
6 - Toga Bike Shop - 110 W End Ave New York, NY 10023
7 - BMW of Manhattan - 555 W 57th St New York, NY 10019
8 - Verizon Wireless - 57 W 57th St New York, NY 10019


In [ ]:
r4m.route